In [22]:
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q trl xformers wandb datasets einops gradio sentencepiece bitsandbytes

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, warnings
import gradio 
from datasets import load_dataset
from trl import SFTTrainer

In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: NVIDIA RTX A6000
Compute Capability: (8, 6)
Total Memory: 51041271808 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 6.2.0-34-generic
Python Version: 3.9.12


In [23]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("lcw99/llama2-ko-chang-13b-instruct-chat")
# model = AutoModelForCausalLM.from_pretrained("lcw99/llama2-ko-chang-13b-instruct-chat")

# Pre trained model
model_name = "beomi/llama-2-ko-7b" 

# Dataset name
dataset_name = "dohun0714/chat_bot"

# Hugging face repository link to save fine-tuned model(Create new repository in huggingface,copy and paste here)
new_model = "dohun0714/llama2_ko_chat"

In [6]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ········
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through g

In [7]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
# dataset["text"][0]
print(dataset)

Dataset({
    features: ['Q', 'A', 'label'],
    num_rows: 11823
})


In [10]:
# Load base model(llama-2-7b-hf) and tokenizer
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit= True,
#     bnb_4bit_quant_type= "nf4",
#     bnb_4bit_compute_dtype= torch.float16,
#     bnb_4bit_use_double_quant= False,
# )
model = AutoModelForCausalLM.from_pretrained(
    model_name,
#     quantization_config=bnb_config,
#     device_map={"": 0}
)
model.to('cuda:0')
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

(True, True)

In [ ]:
# for i, data in enumerate(dataset):
#     print(data)
#     print("\n")
#     if i > 10:  # 첫 10개 요소만 출력
#         break

In [12]:
# 예시 코드
example_sentence = "이것은 테스트 문장입니다."
tokenized_output = tokenizer.tokenize(example_sentence)
print(tokenized_output)

# 토크나이저와 모델의 임베딩 레이어 크기 비교
print(len(tokenizer.vocab))  # 토크나이저 단어장 크기
print(model.model.embed_tokens.weight.size(0)) # 모델 임베딩 레이어 크기

['▁이', '것은', '▁테스트', '▁문', '장', '입니다', '.']
46336
46336


In [14]:
#monitering login
wandb.login(key="ba603a5612d073d6bd76b4b1844ee94b3e05fbbd")
run = wandb.init(project='llama2-kor-chatbot', job_type="training", anonymous="allow")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [15]:
peft_config = LoraConfig(
    lora_alpha= 8,
    lora_dropout= 0.1,
    r= 16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [16]:
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 1,
    per_device_train_batch_size= 8,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 1000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "linear",
    report_to="wandb",
)

In [17]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(46336, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
 

In [21]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 100,
    dataset_text_field="Q",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
) 

Map:   0%|          | 0/11823 [00:00<?, ? examples/s]

In [22]:
# Train model
trainer.train()

Step,Training Loss
30,4.734200
60,5.193900
90,4.315500
120,4.107600
150,4.363000
180,3.647800
210,4.329900
240,3.899900
270,3.901300
300,4.162000


TrainOutput(global_step=739, training_loss=3.961689909030362, metrics={'train_runtime': 297.9927, 'train_samples_per_second': 39.675, 'train_steps_per_second': 2.48, 'total_flos': 4110778157531136.0, 'train_loss': 3.961689909030362, 'epoch': 1.0})

In [24]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

train/epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
train/global_step,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇███
train/learning_rate,▂▃▄▅▆▇███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
train/loss,▆█▅▄▅▂▅▃▃▄▂▃▂▃▄▁▃▂▂▄▁▃▂▂
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0
train/global_step,739


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(46336, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
    

In [59]:
# def stream(user_prompt):
#     runtimeFlag = "cuda:0"
#     system_prompt = 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n'
#     B_INST, E_INST = "### Instruction:\n", "### Response:\n"

#     prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}\n\n{E_INST}"

#     inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

#     streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

#     # Despite returning the usual output, the streamer will also print the generated text to stdout.
#     _ = model.generate(**inputs, streamer=streamer, max_new_tokens=1500)

In [60]:
def stream(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    B_INST, E_INST = "### Instruction:", "### Response:"

    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=1500)

    # Decode and print the generated response
    print(tokenizer.decode(outputs[0]))

stream("나 오늘 기뻐")

<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:나 오늘 기뻐### Response:</s>는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오늘 기뻐.는 오

In [62]:
stream("나 오늘 혼났어")

<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:나 오늘 혼났어### Response:</s>.......?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?!?

In [61]:
stream("안녕 너는 누구야?")

<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:안녕 너는 누구야?### Response:</s> 안녕 나는 �����������야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### Response:나는 너에게 ㅇㅇㅣ야.### Instruction:나는 너에게 ㅇㅇㅣ야.### R

In [31]:
# Clear the memory footprint
del model, trainer
torch.cuda.empty_cache()

In [32]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.float16,
    device_map= {"": 0})
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dohun0714/llama/commit/4759044169af1495cb294c82e6b3af314c4b94b0', commit_message='Upload tokenizer', commit_description='', oid='4759044169af1495cb294c82e6b3af314c4b94b0', pr_url=None, pr_revision=None, pr_num=None)